# Building Realistic Example Data
This __optional bonus assignment__ is worth up to 5% of your final grade. It must be handed in by directly mailing the instructor (Christopher Brooks, brooksch@umich.edu) the assignment no later than Sunday September 27th at 11:59pm EST. This is not an all-or-nothing assignment, partial grades will be provided as appropriate.

## Assignment Overview
A constant need I have when teaching pandas is finding compelling example data to work from. Good example data is real-world, messy enough to need some manipulation, and fits reasonable constraints for a given problem. For instance, if I want to demonstrate joining multiple `DataFrame` together I might want one which is about people and one which is about purchases, where every person has an identifier and a bunch of personal information, and every purchase is linked to a given person. This is much more compelling then a bunch of random `np.ndarray` lists that I create inline while trying to give a lecture!

In addition, I'm taken by domain specific languages, and in this part of the assignment you are required to build a processor for a simple domain specific language I have invented for the purpose of describing pandas `DataFrame` structures! It is expected that you will demonstrate your knowledge of regex here in particular.

Here's an example of the language I've created for this part of the assignment:
```
persons
-------
first_name
last_name*
phone_number
random_number(5) as customer_number [1]*

purchases
---------
isbn10
credit_card_full
random_number(3) as price
random_number(5) as customer_number [1]
```

In this example I describe two `DataFrame` objects by underlying a string with two or more hyphens. The string (`persons` and `purchases`) should be used as the variable name for the `DataFrame` objects created, and the language will always separate multiple `DataFrame` definitions by whitespace and hyphens as shown. Each column in the `DataFrame` is described on its own line with a string (e.g. `first_name*`). The string defines the column as follows:

1. The first word (e.g. `first_name` or `random_number(3)`) describes a function  and optional set of parameters to be called against a common `faker` object (an instance of the `Faker` class) for each entry in the resulting `DataFrame`. For instance, a value of `isbn10` implies that `faker.isbn10()` be called (note the default parameters), while a value of `random_number(5)` implies that `faker.random_number(5)` be called (with my supplied parameters).
2. The first word *may* be followed by some whitespace and then an `as` statement. The `as` statement denotes that the following word be used as the name of the column. For instance, `random_number(3) as price` means that I'm looking for a column named `price` where every instance in the column is of a `faker.random_number(3)` invocation.
3. If there is no `as` statement the name of the column should be the name of the function (with no parameters) I supplied, e.g. `first_name`.
4. The definition may include a reference in the format `[#]` where the # sign is any number. This reference will be used across tables to show that the data in those tables should be of a similar set of values. This is so I can join between tables, which is a common need to demonstrate. In the example given I want each of the tables to have a column called `customer_number` where the data values in the column are such that `set(column1)==set(column2)`. Note that this doesn't mean the columns should be the same (they shouldn't), but just that they should include only 100% overlapping data. See point 6 for data distribution.
5. The sentence may end with a `*`. This indicates that the column described should be made up of unique data (no repeated elements). For instance, you wouldn't want a customer to be accidently assigned the customer number of another person! In the example above I decided I wanted the `last_name` of the persons and their `customer_number` to be unique in that table.
6. By default a column should have 25% repeated data. e.g. something close to `len(column)==len(set(column))*1.25`. This lets me demonstrate operations such as left joins easily.
7. By default, the length of each `DataFrame` created should be 99 items. This is both reasonable for most demonstrations, and a homage to The Great One.
8. The functionality described should be executed in a cell magic function called `%%fakedata`, where the remainder of the cell is the definition in plain text. See https://ipython.readthedocs.io/en/stable/config/custommagics.html for more details.

## An attempt at a more formal grammar

```
function_to_call  ::= <wordcharacters>
parameters        ::= "" | "(" ( wordcharacters | number ) ")"
as_name           ::= "" | "as" <whitespace> <wordcharacters>
column_name       ::= as_name | function_to_call
reference         ::= "" |  "[" number "]"
unique_mark       ::= "" | "*"
column_definition ::= <function_to_call> <parameters> <whitespace> \
                      <as_name> <whitespace> <reference> <unique_mark>
df_sep            ::= "--" ("-"*)
df_definition     ::= <wordcharacters> <newline> <df_sep> <newline> \
                      (<column_definition>*) <newline> <newline>
language_spec     ::= <def_definition>*
```


## Background: What is this `Faker` class?
The `Faker` class defines a number of great functions that generate realistic data. The way it works is that you create a new instance of `Faker` with no parameters, then call various methods on that object which are predefined at https://faker.readthedocs.io/en/stable/

This is demonstrated below showing a single entry into a `DataFrame` using the description above.

In [4]:
import pandas as pd
from faker import Faker

fake = Faker()
person = pd.DataFrame( [{"first_name": fake.first_name(),
                        "last_name": fake.last_name(),
                        "phone_number": fake.phone_number(),
                        "customer_number": fake.random_number(5)}])
person

,first_name,last_name,phone_number,customer_number
0,James,Foster,232-492-6132x0537,48749
